# Podcast Episode Erstellen

Zuächst Laden wir uns alle Datein 

In [7]:
from db_connect import db_get_df

df = db_get_df(table="transcript_segments")

print(df.dtypes)

filename           object
segment_text       object
start             float64
end               float64
segment_id          int64
embedding_json     object
dtype: object


In [7]:
print(len(df))
print(df.sample(2).to_markdown())

3114
|      | filename                                            | segment_text                                                                            |   start |     end |   segment_id | embedding_json                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

Play random Audio

In [1]:
from pydub import AudioSegment
from pydub.playback import play
from pydub import AudioSegment
from playsound import playsound
import IPython 
import io
from db_connect import db_get_df
from segment_ranking.rank_segments import get_most_similar_documents
from Audio_segmentation.split_audio import produce_snippets
from Audio_segmentation.concat_audio import produce_audio

In [2]:
df = db_get_df(table="transcripts")

Demonstration

In [9]:
userInput = "Auf dem l Feiern und trinken"

best_fitting = get_most_similar_documents(userInput, 7)
print(best_fitting["segment_text"].to_markdown())

produce_snippets()
produce_audio()


saved
|      | segment_text                                                                                           |
|-----:|:-------------------------------------------------------------------------------------------------------|
|  586 | Einer gegen viele, zwei gegen zwei, alle gegen alle.                                                   |
|  160 | hervorgebracht habe und zugleich das widerspenstigste und ungelehrigste, boshafteste und arglistigste. |
|  965 | rund drei Millionen Sozialwohnungen. Inzwischen sind es nur noch etwa ein Drittel, gut eine            |
| 1689 | interview zu werden. Von Golda Meir über Indira Gandhi, Muammar al-Gaddafi oder Willy                  |
|  329 | Medikamente dabei. Paracetamol, Ibuprofen, Basisantibiotika, Penicillin natürlich.                     |
|  711 | Das ist ein sehr wichtiges Thema.                                                                      |
|  712 | Das ist ein sehr wichtiges Thema.                                        

In [8]:
IPython.display.Audio("concatenated_audio.mp3")